In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import sys, os
cur_dir = os.getcwd()
test_dir = os.path.join(cur_dir, os.pardir)
src_dir = os.path.join(test_dir, os.pardir)
sys.path.append(os.path.abspath(src_dir))

In [4]:
!pip install torchvision

     ---------------------------------------- 1.2/1.2 MB 15.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from data.processed.CelebA_1024.call_celeba import CelebALoader
cl = CelebALoader()

In [4]:
df = cl.load_anno('list_attr_celeba_1024.txt')

In [5]:
df

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
001020.jpg,1,1,1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,1,-1,-1,-1,-1,1,1
001021.jpg,-1,-1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,1,-1,1,-1,-1,1
001022.jpg,1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
001023.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,-1,1,-1,-1,1,-1,-1,1


In [6]:
len(df.loc[df['Bald']==1])

22

In [7]:
len(df.loc[df['Wearing_Necklace']==1])+len(df.loc[df['Wearing_Earrings']==1])
#귀금속 착용여부 판단도 좋을 것 같다. x-ray찍거나 놀이기구 탈때 귀금속 착용하면 안되므로 경고 문구 띄워줄 수도 있을듯..

341

In [8]:
y = df['Bald'].replace(to_replace=-1, value=0)
index_bald = np.where(np.array(y)==0)


In [9]:
index_bald

(array([   0,    1,    2, ..., 1021, 1022, 1023], dtype=int64),)

In [10]:
y=torch.tensor(y.values)


In [24]:
y=y.view(-1,1)

In [25]:
y

tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])

In [12]:
imgs = cl.load_img()
imgs[0].size()

torch.Size([218, 178])

In [13]:
len(imgs)

1024

In [14]:
imgs

[tensor([[0.8863, 0.8863, 0.8863,  ..., 0.9124, 0.9333, 0.9333],
         [0.8863, 0.8863, 0.8863,  ..., 0.9176, 0.9333, 0.9333],
         [0.8863, 0.8863, 0.8863,  ..., 0.9255, 0.9373, 0.9373],
         ...,
         [0.3137, 0.2157, 0.3359,  ..., 0.2680, 0.2719, 0.2719],
         [0.2706, 0.3020, 0.4144,  ..., 0.2444, 0.2523, 0.2523],
         [0.4196, 0.5608, 0.7242,  ..., 0.2444, 0.2536, 0.2536]]),
 tensor([[0.2144, 0.2065, 0.2065,  ..., 0.2837, 0.3046, 0.3124],
         [0.2144, 0.2065, 0.2065,  ..., 0.2837, 0.3046, 0.3124],
         [0.2170, 0.2065, 0.2065,  ..., 0.2837, 0.3046, 0.3124],
         ...,
         [0.8405, 0.8797, 0.8771,  ..., 0.3987, 0.4758, 0.5190],
         [0.8993, 0.8797, 0.8510,  ..., 0.4444, 0.4824, 0.4824],
         [0.8993, 0.8797, 0.8510,  ..., 0.4562, 0.4837, 0.4837]]),
 tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.

In [15]:
type(imgs)

list

In [16]:
for i in range(1024):
    imgs[i].unsqueeze_(dim=0)
imgs[0].shape

torch.Size([1, 218, 178])

In [17]:
imgs[0]

tensor([[[0.8863, 0.8863, 0.8863,  ..., 0.9124, 0.9333, 0.9333],
         [0.8863, 0.8863, 0.8863,  ..., 0.9176, 0.9333, 0.9333],
         [0.8863, 0.8863, 0.8863,  ..., 0.9255, 0.9373, 0.9373],
         ...,
         [0.3137, 0.2157, 0.3359,  ..., 0.2680, 0.2719, 0.2719],
         [0.2706, 0.3020, 0.4144,  ..., 0.2444, 0.2523, 0.2523],
         [0.4196, 0.5608, 0.7242,  ..., 0.2444, 0.2536, 0.2536]]])

In [18]:
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.init
import torch.nn.functional as F 

In [26]:
class CNN(nn.Module):
    def __init__(self):
        
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1,20,5,1) 
        self.conv2 = nn.Conv2d(20,10,5,1)
        self.pool = nn.MaxPool2d(2) 
         
        self.combination = nn.Sequential(
                           nn.Linear(2091, 64),#dense1
                           nn.ReLU(), 
                           nn.Linear(64, 2), #dense2
                           )


    def forward(self, x):
       
        x = self.conv1(x)
       
        x = F.relu(x) 
        x = self.pool(x)       
        x = self.conv2(x)       
        x = F.relu(x)
        x = self.pool(x)
        
        
        x= x.view(x.size(0),-1) 
        
        x = self.combination(x)
        
        return F.log_softmax(x , dim = 1)

In [20]:
model = CNN()
learning_rate =0.001
#batch_size = 64
epochs = 15

In [21]:
lossfunc = nn.CrossEntropyLoss()

In [22]:
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [27]:
for epoch in range(epochs):
    train_loss = 0

    for i in range(1024):
        x = imgs[i]
        label = y[i]
        
        optimizer.zero_grad()

        prediction = model.forward(x)
        
        
        
        loss = lossfunc(prediction,label)

        loss.backward()
        
        optimizer.step()
        
        train_loss += loss
         
    with torch.no_grad(): 
        correct = 0
        total = 1024
        bald_correct = 0

        for i in range(1024):
                        
            pred = torch.argmax(model(x),dim=1)
            label = y[i]
            if label==pred :
                correct += 1
                if i in index_bald:
                    bald_correct+=1
                    
        
            
            
    print("[Epoch %d] loss: %.3f, acc: %.2f, bald_acc :%2f" \
        % (epoch + 1, train_loss / 1024, 100.*correct/total,100*bald_correct/22))
    train_loss = 0 

ValueError: Expected input batch_size (10) to match target batch_size (1).